In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
df = pd.read_excel('..\\ny_crime_13.xls', header=4)
df.head(n=10)

In [ ]:
df_features = pd.DataFrame()
df_features['pop'] = df['Population']
df_features['pop_squared'] = df_features['pop']**2
df_features['murder_bin'] = np.where(df.iloc[:, 3] > 0, 1, 0)
df_features['murder'] = df.iloc[:,3]
df_features['robbery_bin'] = np.where(df.iloc[:, 6] > 0, 1, 0)
df_features['robbery'] = df.iloc[:, 6]
df_features['theft_bin'] = np.where(df['Larceny-\ntheft'] > 0, 1, 0)
df_features['theft'] = df['Larceny-\ntheft']
df_features['prop_crime'] = df.loc[:, 'Property\ncrime']
df_features['violent'] = df['Violent\ncrime']
df_features.dropna(inplace=True)
df_features.head()

In [ ]:
# Restructure the data so we can use facetgrid
df_long = df_features
df_long = pd.melt(df_long.drop(['murder', 'robbery', 'theft', 'theft_bin'], axis=1),
                  id_vars=['murder_bin', 'robbery_bin'])
g = sns.FacetGrid(df_long, col='variable', size = 5, aspect=.7, sharey=False)
g = g.map(sns.boxplot, 'murder_bin', 'value', showfliers=False)
plt.show()

g2 = sns.FacetGrid(df_long, col='variable', size = 5, aspect=.7, sharey=False)
g2 = g2.map(sns.boxplot, 'robbery_bin', 'value', showfliers=False)
plt.tight_layout()
plt.show()

These boxplots show that places that have murder and robbery occur more in places with higher population. The last column of boxplots shows that property crime is higher in places with robbery and murder, but that is most likely becasue areas of high crime are grouped together.

In [ ]:
regr = linear_model.LinearRegression()
Y = df_features['prop_crime'].values.reshape(-1,1)
parameters = ['theft', 'robbery', 'murder', 'pop', 'pop_squared']
X = df_features[parameters]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)
regr.fit(X_train,y_train)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared on the training set:')
print(regr.score(X_train, y_train))
print('\nR-squared on the test set:')
print(regr.score(X_test, y_test))

These are pretty high r-squared values, but since they are both pretty high, I'd say that we are ok.

In [ ]:
y_pred = regr.predict(X_test)
residual = y_test - y_pred
plt.hist(residual, bins=40)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
# plt.xlim([-500, 500])
plt.show()

I still do not have enough experience to know if this is considered normal or not... It almost appears that it has a left tail, and a strange bump thing on the right. Could this mean that a variable is missing, and could explain the two little bumps on either side?

Or perhaps these are just outliers?

In [ ]:
correlation_matrix = X.corr()
display(correlation_matrix)

When using binary categorical variables, they are lowly correlated with each other. However, as we saw in the box plots, the number of crimes committed is highly correlated with the population. Both methods give a pretty reasonably good classifier.

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(y_pred, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
# plt.xlim([-200, 1000])
plt.show()



Interesting. I have played around with switching the features between binary and actual values. Using the raw values produces a better classifier with a tighter cluster of residuals at lower estimates. Our classifier seems to fail at the higher predictions, and I think this may be due to the fact that we have much less data for higher populations. Perhaps we could classify these predictions as outliers, and get rid of the data that caused them? If that's the case, then we would need to be careful to only use our classifier with data that fits into the same domain as the predictions we kept.

# Evaluating the Classifier
_From here on, everything will be new content. All of the above was copied from a previous assignment._

The first thing we should do is run a whole model F-test. I will do this by hand for my understanding.

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
ssef = ((df_features.prop_crime.values - y_pred)**2).sum()
sser = ((df_features.prop_crime - df_features.prop_crime.mean())**2).sum()
n = len(df_features.prop_crime)
pf = len(parameters)
pr = 1
dff = n - pf
dfr = n - pr

f_test = ((ssef-sser)/(dff - dfr))/(ssef/dff)
f_test

It is unclear at this point what I should do with a p-value of $-84.5$. I am fairly sure that this means my model is significant, but then again I'm unsure because I've never seen a negative p-value, and the f-test distribution itself is never negative either. So... Good to go?

In [ ]:
linear_formula = 'prop_crime ~ theft+robbery+murder+pop+pop_squared'

# Now fit the model using a different module then before so we can pull out the
# individual p-values of significance for each parameter
lin_mod = smf.ols(formula=linear_formula, data=df_features).fit()

In [ ]:
lin_mod.params

In [ ]:
lin_mod.pvalues

Let's add another feature, but we will add it to a separate model. I want to see how an additional feature will affect the generalization to another dataset.

In [ ]:
regr2 = linear_model.LinearRegression()
Y = df_features['prop_crime'].values.reshape(-1,1)
parameters = ['theft', 'robbery', 'murder', 'pop', 'pop_squared', 'violent']
X = df_features[parameters]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)
regr2.fit(X_train,y_train)

# Inspect the results.
print('\nCoefficients: \n', regr2.coef_)
print('\nIntercept: \n', regr2.intercept_)
print('\nR-squared on the training set:')
print(regr2.score(X_train, y_train))
print('\nR-squared on the test set:')
print(regr2.score(X_test, y_test))

In [ ]:
y_pred = regr2.predict(X_test)
residual = y_test - y_pred
plt.hist(residual, bins=40)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
# plt.xlim([-500, 500])
plt.show()

In [ ]:
correlation_matrix = X.corr()
display(correlation_matrix)

From my discussion with Katherine, high multi-collinearity results in an unstable model. This manifests itself in coefficients that are different every time we run the model. However, I'm operating on the technicality that the assignment originally asked for a model that explains the most variance possible, rather than understanding the mechanisms. So, with that in mind, I'm going to move on even though these are all really highly correlated!

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(y_pred, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
# plt.xlim([-200, 1000])
plt.show()

This got better as well, and I'm pretty pleased with that. I have a gut feeling that we are overfitting, becasue that r-value is extremely high. Let's find out with a new data set!

In [ ]:
# Load in a brand new dataset. I chose the california crime dataset, also from 2013.
df_ca = pd.read_excel('..\\ny_crime_13.xls', header=4)
df_ca.head(n=5)

In [ ]:
# Create the same features that we made for the NY dataset
df_ca_features = pd.DataFrame()
df_ca_features['pop'] = df['Population']
df_ca_features['pop_squared'] = df_ca_features['pop']**2
df_ca_features['murder'] = df.iloc[:,3]
df_ca_features['robbery'] = df.iloc[:, 6]
df_ca_features['theft'] = df['Larceny-\ntheft']
df_ca_features['prop_crime'] = df.loc[:, 'Property\ncrime']
df_ca_features['violent'] = df['Violent\ncrime']
df_ca_features.dropna(inplace=True)
df_ca_features.head()

In [ ]:
# We will treat the entire dataset as a test set, because we want to see
# how our model generalizes to a different dataset.
Y_ca = df_ca_features['prop_crime'].values.reshape(-1,1)
parameters = ['theft', 'robbery', 'murder', 'pop', 'pop_squared', 'violent']
X_ca = df_ca_features[parameters]

print('\nR-squared on the test set:')
print(regr2.score(X_ca, Y_ca))

Wow. This model still has an extremely high $R^2$ value. I'm still really hesitant to say that we are doing well, but then again maybe this dataset was chosen specifically for its ease of model fitting. I just can't say at this point!

In [ ]:
y_pred = regr2.predict(X_ca)
residual = Y_ca - y_pred
plt.hist(residual, bins=40)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
# plt.xlim([-500, 500])
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(y_pred, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
plt.xlim([-200, 13000])
plt.show()

This seems to be fine too, I think. Turning it in.